In [1]:
#загрузка библиотек
import sklearn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

#Кластеризация
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture
import matplotlib as mpl
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

%matplotlib inline

https://digitrain.ru/articles/13812/
    
https://pythonru.com/uroki/sklearn-kmeans-i-knn

https://www.dominodatalab.com/blog/getting-started-with-k-means-clustering-in-python

Парсинг

In [2]:
df_us = pd.read_csv('', delimiter=';', low_memory=False)
df_us

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
df_ope = pd.read_csv('', delimiter=';', low_memory=False)
df_ope

Формирование датасетов

In [ ]:
df_us[''] = pd.to_numeric(df_us[''], errors='coerce').astype('Int64')
df_ope[''] = pd.to_numeric(df_ope[''], errors='coerce').astype('Int64')

In [ ]:
plt.figure(figsize=(23, 15))
sns.heatmap(df_us.corr(), annot = True, vmin=-1, vmax=1,cmap= 'coolwarm')

На корреляционной карте выше видно, что довольно сильно коррелируют между собой атрибуты:



In [ ]:
df = df_us[['', '']].head(1000)
df

Обработка данных

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)


In [ ]:
#Вывод коробчатых диаграм в цикле только по числовым атриубтам
for col in df.select_dtypes(exclude=["object", "datetime64[ns]"]).columns:
    #построение фигуры
    plt.figure()
    #Заголовк
    plt.title(col)
    #Вывод диаграммы
    sns.boxplot(data=df[col])
    plt.show()

In [ ]:
#Очистка выбросов
for col in df.select_dtypes(exclude=["object", "datetime64[ns]"]).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 

    filter = (df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 *IQR)
    df = df.loc[filter]  

In [ ]:
#Вывод коробчатых диаграм в цикле только по числовым атриубтам
for col in df.select_dtypes(exclude=["object", "datetime64[ns]"]).columns:
    #построение фигуры
    plt.figure()
    #Заголовк
    plt.title(col)
    #Вывод диаграммы
    sns.boxplot(data=df[col])
    plt.show()

Доп атр

In [ ]:
df[""] = df[""].map(df_ope.groupby("")[""].sum())

In [ ]:
df

Кластеризация

Переменная для атрибутов и прочее

In [ ]:
#Помещу их в переменную X
X = df[["", "", ""]]

In [ ]:
#обьявляю MinMaxScaler
scaler = MinMaxScaler()
#преобразую данные
X = scaler.fit_transform(X)

In [ ]:
#Обьявляю метод главных компонент
pca = PCA(n_components=2)
#Применяю его на данных
X = pca.fit_transform(X)

KMeans

Определение количества кластров методом локтя. Метод локтя строит график суммы квадратов внутри кластера (WCSS) в зависимости от количества кластеров. 

In [ ]:
#Нам нужно определить цикл for, который содержит экземпляры класса KMeans. Этот цикл for будет перебирать номера кластеров с первого по 10
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(X)

In [ ]:
#Затем мы добавляем значения WCSS в наш список. Мы получаем доступ к этим значениям через атрибут инерции объекта KMeans:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

In [ ]:
#Наконец, мы можем построить график зависимости WCSS от количества кластеров. Во-первых, давайте импортируем Matplotlib and Seaborn, который позволит нам создавать и форматировать визуализации данных:
sns.set()
plt.plot(range(1, 11), wcss)
plt.title('‘Selecting the Number of Clusters using the Elbow Method’')
plt.xlabel('‘Clusters’')
plt.ylabel('‘WCSS’')
plt.show()

Из этого графика мы видим, что четыре - оптимальное количество кластеров, поскольку именно здесь появляется «изгиб» кривой.

In [ ]:
#Настройка параметров
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
#Предсказание
kmpreds = kmeans.predict(X)
#Заношу кластеризированные метки в набор данных
df["KMCLUSTS"] = kmpreds


In [ ]:
customcmap = ListedColormap(["crimson", "mediumblue", "darkmagenta", "pink"])

fig, ax = plt.subplots(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=df["KMCLUSTS"], 
 cmap = customcmap)
ax.set_xlabel(r'x', fontsize=14)
ax.set_ylabel(r'y', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
#Использую метрику calinski_harabasz_score
print(sklearn.metrics.calinski_harabasz_score(X, df["KMCLUSTS"]))
#Использую метрику davies_bouldin_score
print(sklearn.metrics.davies_bouldin_score(X, df["KMCLUSTS"]))

MiniBatchKMeans

In [ ]:
#Настройка параметров
kbmeans = MiniBatchKMeans(n_clusters=4, random_state=0, batch_size=4096)
#Заношу кластеризированные метки в набор данных
df["KBCLUSTS"] = kbmeans.fit_predict(X)

In [ ]:
customcmap = ListedColormap(["crimson", "mediumblue", "darkmagenta", "pink"])

fig, ax = plt.subplots(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=df["KBCLUSTS"], 
 cmap = customcmap)
ax.set_xlabel(r'x', fontsize=14)
ax.set_ylabel(r'y', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
#Использую метрику calinski_harabasz_score
print(sklearn.metrics.calinski_harabasz_score(X, df["KBCLUSTS"]))
#Использую метрику davies_bouldin_score
print(sklearn.metrics.davies_bouldin_score(X, df["KBCLUSTS"]))

GaussianMixtures

In [ ]:
#Настройка параметров
gm = GaussianMixture(n_components=2, random_state=0).fit_predict(X)
#Заношу кластеризированные метки в набор данных
df["GMCLUSTS"] = gm

In [ ]:
customcmap = ListedColormap(["crimson", "mediumblue", "darkmagenta", "pink"])

fig, ax = plt.subplots(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=df["GMCLUSTS"], 
 cmap = customcmap)
ax.set_xlabel(r'x', fontsize=14)
ax.set_ylabel(r'y', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
#Использую метрику calinski_harabasz_score
print(sklearn.metrics.calinski_harabasz_score(X, df["GMCLUSTS"]))
#Использую метрику davies_bouldin_score
print(sklearn.metrics.davies_bouldin_score(X, df["GMCLUSTS"]))

Итог

Решение

По результатам метрик и визуализаций, лучшим алгоритмом является GussianMixture

In [ ]:
# Удаление ненужных атрибутов с кластерами
df.drop(columns=['KMCLUSTS', 'KBCLUSTS'], axis=1, inplace=True)

Сохранение

In [ ]:
#df.to_csv('output.csv', index=False, encoding='utf-8-sig')

Счеты

lan

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Алгоритмы классификации
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Игнорирование ошибок будущих версий
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

Разведочный анализ

In [ ]:
# df = pd.read_csv('output.csv')

In [ ]:
df

In [ ]:
skip = ['AGE', 'INCOME_MAIN_AMT', 'ID', 'MM_W_CARD', 'MM_IN_BANK', 'SUM_TRANS']
#Категориальные признаки
for i in df0.columns:
    if i not in skip and i != 'REG_CODE':
        sns.catplot(x=i, kind='count', palette='ch:20', data=df)
        plt.show()

In [ ]:
sns.catplot(x='', kind='count', palette='ch:20', height=4, aspect=5, data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
newdf = df[['','']].groupby('REG_CODE').sum()
newdf.plot(kind='bar')

In [ ]:
newdf = df[['','']].groupby('GENDER').sum()
newdf.plot(kind='bar')

Разбиение выборки

In [ ]:
df.head()

Система рекомендаций будет определять категорию принадлежности, то есть порекомендовать пользователю товар для мужчин или для женщин

In [ ]:
#обьявляю MinMaxScaler
scaler = MinMaxScaler()
#преобразую данные
X = scaler.fit_transform(X)

In [ ]:
#Помещу их в переменную X
X = df[["AGE", "EDUCATION_LEVEL", "REG_CODE", "GMCLUSTS"]]
y = df["GENDER"]

In [ ]:
#Разделение данных на обучающий и тестовый набор
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Реализация моделей

KNN

In [ ]:
#Объявление классификатор
neigh = KNeighborsClassifier()
#Обучение классификатора
neigh.fit(X_train, y_train)

GaussianNB

In [ ]:
#Объявление классификатор
gnb = GaussianNB()
#Обучение классификатора
gnb.fit(X_train, y_train)
GaussianNB()

LogisticRegression

In [ ]:
#Объявление классификатор
clf = LogisticRegression()
#Обучение классификатора
clf.fit(X_train, y_train)

Метрики
Теперь посмотрю на результаты метрик, я буду рассматривать метрики - f1 и accuracy sore, f так же f1 по macro avg

KNN

In [ ]:

#Вывожу результаты метрик при помощи classification_report
print(classification_report(y_test, neigh.predict(X_test)))

GaussianNB

In [ ]:
#Вывожу результаты метрик при помощи classification_report
print(classification_report(y_test, gnb.predict(X_test)))

LogisticRegression

In [ ]:
#Вывожу результаты метрик при помощи classification_report
print(classification_report(y_test, clf.predict(X_test)))

Решение
По итогам метрик, остановлюсь на алгоритме KNN, так как он имеет самые высокие показатели, по f1 accuracy показал 0.99, и f1 macro avg 0.99. В то время как остальные алгоритмы показали 1, что может оказаться переобучением.

Оптимизация модели

In [ ]:
df.head()

In [ ]:
#обьявляю MinMaxScaler
scaler = MinMaxScaler()
#преобразую данные
X = scaler.fit_transform(X)

In [ ]:
#Помещу их в переменную X
X = df[["AGE", "EDUCATION_LEVEL", "REG_CODE", "GMCLUSTS", 'INCOME_MAIN_AMT']]
y = df["GENDER"]

In [ ]:
#Разделение данных на обучающий и тестовый набор
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Реализация моделей

In [ ]:
#Объявление классификатор
neigh2 = KNeighborsClassifier(n_neighbors=2, leaf_size=10, p=3, n_jobs=-1)
#Обучение классификатора
neigh2.fit(X_train, y_train)

Метрика

In [ ]:
#Вывожу результаты метрик при помощи classification_report
print(classification_report(y_test, neigh2.predict(X_test)))

Модель стала хуже, так что для дальнейшего использования лучше будет оставить первую версию

Сохранение лучшей модели

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(neigh, open(filename, 'wb'))

загрузка модели

In [ ]:
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# df.to_csv('output2.csv', index=False, encoding='utf-8-sig')